In [2]:
import sys
print(sys.executable)
print(sys.version)

import platform
print(platform.platform())
print(platform.machine())



/opt/miniconda3/envs/intsys1/bin/python
3.14.3 | packaged by Anaconda, Inc. | (main, Feb 24 2026, 22:43:19) [Clang 20.1.8 ]
macOS-26.3-arm64-arm-64bit-Mach-O
arm64


In [3]:
import sys, subprocess

subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip", "setuptools", "wheel"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "gurobipy"])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.8 MB/s  0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 80.10.2
    Uninstalling setuptools-80.10.2:
      Successfully uninstalled setuptools-80.10.2


0

In [4]:
import gurobipy as gp
from gurobipy import GRB

print("Gurobi version:", gp.gurobi.version())


Gurobi version: (13, 0, 1)


In [5]:
# --- Data ---
capacity = 10
items = {
    "A": {"weight": 6, "value": 30},
    "B": {"weight": 3, "value": 14},
    "C": {"weight": 4, "value": 16},
    "D": {"weight": 2, "value": 9},
    "E": {"weight": 5, "value": 20},
}

# --- Model ---
m = gp.Model("knapsack")

# Decision variables: x[i] = 1 if item i is taken
x = m.addVars(items.keys(), vtype=GRB.BINARY, name="x")

# Objective: maximize total value
m.setObjective(gp.quicksum(items[i]["value"] * x[i] for i in items), GRB.MAXIMIZE)

# Capacity constraint
m.addConstr(gp.quicksum(items[i]["weight"] * x[i] for i in items) <= capacity, name="capacity")

# Solve
m.optimize()

# --- Results ---
if m.status == GRB.OPTIMAL:
    chosen = [i for i in items if x[i].X > 0.5]
    total_value = sum(items[i]["value"] for i in chosen)
    total_weight = sum(items[i]["weight"] for i in chosen)
    print("Optimal chosen items:", chosen)
    print("Total value:", total_value)
    print("Total weight:", total_weight)
else:
    print("No optimal solution found. Status code:", m.status)


Restricted license - for non-production use only - expires 2027-11-29
Gurobi Optimizer version 13.0.1 build v13.0.1rc0 (mac64[arm] - Darwin 25.3.0 25D125)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 1 rows, 5 columns and 5 nonzeros (Max)
Model fingerprint: 0xb37c3a31
Model has 5 linear objective coefficients
Variable types: 0 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [2e+00, 6e+00]
  Objective range  [9e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]

Found heuristic solution: objective 44.0000000
Presolve removed 1 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 10 available processors)

Solution count 2: 46 44 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.600000000000e+01, best bound 4.600000000000

In [5]:
import gurobipy as gp
from gurobipy import GRB

# ----------------------------
# Data (scores to maximize)
# ----------------------------
students = ["S1", "S2", "S3", "S4"]
topics = ["A", "B", "C", "D"]

score = {
    ("S1","A"): 50, ("S1","B"): 49, ("S1","C"):  1, ("S1","D"):  1,
    ("S2","A"): 48, ("S2","B"):  2, ("S2","C"): 47, ("S2","D"):  1,
    ("S3","A"):  2, ("S3","B"): 46, ("S3","C"): 45, ("S3","D"):  1,
    ("S4","A"):  1, ("S4","B"):  1, ("S4","C"):100, ("S4","D"): 44,
}

# ----------------------------
# Model: Maximize total score
# ----------------------------
m = gp.Model("assignment_max_score")

# x[s,t] = 1 if student s is assigned topic t
x = m.addVars(students, topics, vtype=GRB.BINARY, name="x")

# Objective: maximize total score
m.setObjective(gp.quicksum(score[s, t] * x[s, t] for s in students for t in topics), GRB.MAXIMIZE)

# Each student gets exactly one topic
m.addConstrs((gp.quicksum(x[s, t] for t in topics) == 1 for s in students), name="one_topic_per_student")

# Each topic assigned to exactly one student
m.addConstrs((gp.quicksum(x[s, t] for s in students) == 1 for t in topics), name="one_student_per_topic")

# Solve
m.optimize()

# ----------------------------
# Results
# ----------------------------
if m.status == GRB.OPTIMAL:
    assignment = []
    total = 0
    for s in students:
        for t in topics:
            if x[s, t].X > 0.5:
                assignment.append((s, t, score[s, t]))
                total += score[s, t]

    print("\nOptimal assignment (maximize total score):")
    for s, t, sc in assignment:
        print(f"  {s} -> {t}  (score {sc})")
    print("Total score:", total)
else:
    print("No optimal solution found. Status code:", m.status)


Gurobi Optimizer version 13.0.1 build v13.0.1rc0 (mac64[arm] - Darwin 25.2.0 25C56)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 8 rows, 16 columns and 32 nonzeros (Max)
Model fingerprint: 0x018fa18d
Model has 16 linear objective coefficients
Variable types: 0 continuous, 16 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Found heuristic solution: objective 139.0000000
Presolve time: 0.00s
Presolved: 8 rows, 16 columns, 32 nonzeros
Variable types: 0 continuous, 16 integer (16 binary)

Root relaxation: objective 1.980000e+02, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     198.0000000  198.00000  0.00

In [6]:
import gurobipy as gp
from gurobipy import GRB

# --- Salad mixing LP (grams) ---
# Decision variables:
#   x = grams of tomatoes
#   y = grams of lettuce
#
# Data:
#   cost: tomatoes 0.006 €/g, lettuce 0.002 €/g
#   quality: tomatoes 9 units/g, lettuce 3 units/g
#
# Constraints:
#   x + y >= 300        (at least 300g of salad)
#   9x + 3y >= 1800     (minimum total quality)
#   x <= 250            (max tomatoes, to avoid "all tomato")
#   x, y >= 0
#
# Objective:
#   minimize total cost

c_tom, c_let = 0.006, 0.002
q_tom, q_let = 9, 3

W_min = 300
Q_min = 1800
x_max = 250

m = gp.Model("SaladMixing")

x = m.addVar(lb=0, name="tomatoes_g")
y = m.addVar(lb=0, name="lettuce_g")

m.setObjective(c_tom * x + c_let * y, GRB.MINIMIZE)

m.addConstr(x + y == W_min, name="min_weight")
m.addConstr(q_tom * x + q_let * y >= Q_min, name="min_quality")
m.addConstr(x >= x_max, name="max_tomatoes")

m.optimize()

if m.Status == GRB.OPTIMAL:
    print("OPTIMAL")
    print(f"Tomatoes: {x.X:.2f} g")
    print(f"Lettuce:  {y.X:.2f} g")
    print(f"Total weight: {x.X + y.X:.2f} g")
    print(f"Total quality: {q_tom*x.X + q_let*y.X:.2f}")
    print(f"Total cost: €{m.ObjVal:.4f}")
elif m.Status == GRB.INFEASIBLE:
    print("INFEASIBLE")
else:
    print("Status:", m.Status)


Gurobi Optimizer version 13.0.1 build v13.0.1rc0 (mac64[arm] - Darwin 25.3.0 25D125)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 3 rows, 2 columns and 5 nonzeros (Min)
Model fingerprint: 0x1cb8aa58
Model has 2 linear objective coefficients
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [2e-03, 6e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 2e+03]

Presolve removed 3 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.600000000e+00
OPTIMAL
Tomatoes: 250.00 g
Lettuce:  50.00 g
Total weight: 300.00 g
Total quality: 2400.00
Total cost: €1.6000
